### Writing fire coefficient data from FVS output
Decision was made on 6/22/2016 to reparameterize the multi-objective model for the Drink area using a different proxy for fire hazard. Instead of using the value of the Anderson fuel model (1-13), using that used in the work by Svetlana Kushch Schroder. Her metric was modified to include values for 2 additional fuel models: 4 and 9 with respective fire hazard ratings of 5 and 2.

In [129]:
import pandas as pd
import numpy as np

In [148]:
# Prepare a dataframe to serve as the output


A = ["Ensemble_rcp45","Ensemble_rcp85","none"] # climate scenarios considered
R = ["trtInNeither","trtIn1","trtIn2","trtInBoth"] # the possible prescriptions

fms = pd.read_csv("../data/fuelModels_hazardRatings.csv",index_col=0) # lookups for fuel model -> hazard rating

fire = pd.read_csv("../data/fire_test.csv") # an existing dummy file
fire = fire.loc[fire["climateScenario"].isin(A)] # remove unused climate scenarios
fire["stand"] = fire["stand"].astype(int)
fire = fire.set_index(fire["stand"].astype(str) + "-" + fire["climateScenario"])
fire[[r for r in R]] = 0.0 # set all values to the default fire hazard rating

In [149]:
# Read in the fuel model output stands' area info

areas = pd.read_csv("../data/standAreas.csv",index_col=0)
eohFMs = pd.read_csv("../data/standsFMData_2095.csv")
eohFMs = eohFMs.replace(to_replace=["NONE","E85","E45"],value=["none","Ensemble_rcp85","Ensemble_rcp45"])
eohFMs["Stand"] = eohFMs["Stand_GNN_PAG"].map(lambda x:int(x[0:x.index("_")]))
eohFMs["GNN"] = eohFMs["Stand_GNN_PAG"].map(lambda x:int(x[x.index("_")+1:x.rfind("_")]))
eohFMs["TEMFM1"] = eohFMs["TEMFM1"].map(lambda x:fms.loc[x,"Hazard Rating"])

In [150]:
# Store counts of GNNs in Stands

gnns = pd.read_json("../data/gnnsInStands.json").fillna(0)
gnns.columns = gnns.columns.map(lambda x:int(x))
totalGnnsPerStand = gnns.ix[len(gnns)-1]
#gnns.loc[str(-106),10]
#totalGnnsPerStand[236]

In [151]:
#areas.loc[0,"area (ha)"]

In [152]:
for stand in areas.index:
    for cs in A:
        for r in R:
            df = eohFMs.loc[((eohFMs["Stand"]==stand) & \
                             (eohFMs["TreatmentAssignment"]==r)) &\
                           (eohFMs["ClimateScenario"]==cs)]
            
            # for each GNN in this df
                # calculate what pct of the total gnn count for this stand it is
                # multiply that by the total area of the stand
                # multiply that by its fire hazard rating
            # sum
            standArea = areas.loc[stand,"area (ha)"]
                
            areaOfGnnsInStand = [standArea*gnns.loc[str(gnn),stand]/totalGnnsPerStand[stand] for gnn in df["GNN"]]
            
            hazardContrib = sum(areaOfGnnsInStand*df["TEMFM1"])

            fire.set_value(str(stand)+"-"+cs,r,hazardContrib)
        

In [157]:
#sum(areaOfGnnsInStand) - areas.loc[302]
sum(areaOfGnnsInStand*df["TEMFM1"])
cs

'none'

In [167]:
fire.rename(columns={"trtInNeither":"none","trtIn1":"first","trtIn2":"second","trtInBoth":"both"})\
.replace(to_replace={"climateScenario": {"none":"NONE"}})\
.to_csv("../data/fire_20160624.csv",index=None)
#fire.to_csv("../data/fire_20160624.csv",index=None)
fire

,stand,climateScenario,trtInNeither,trtIn1,trtIn2,trtInBoth
0-none,0,none,32.800554,25.685467,28.318049,26.539277
0-Ensemble_rcp45,0,Ensemble_rcp45,31.164084,28.175747,28.887256,28.318049
0-Ensemble_rcp85,0,Ensemble_rcp85,31.092933,29.527614,28.104596,28.531502
1-none,1,none,99.767842,87.993997,72.059471,77.636555
1-Ensemble_rcp45,1,Ensemble_rcp45,94.190758,91.535003,89.587450,89.321874
1-Ensemble_rcp85,1,Ensemble_rcp85,92.420255,92.420255,85.692343,84.807092
2-none,2,none,387.637545,331.217851,353.028660,310.398444
2-Ensemble_rcp45,2,Ensemble_rcp45,375.740741,335.093326,342.483765,329.325178
2-Ensemble_rcp85,2,Ensemble_rcp85,374.749340,326.801613,343.745547,332.209252
3-none,3,none,269.346693,290.840469,261.522599,270.605743
